In [1]:
import argparse
import h2o
from h2o.automl import H2OAutoML, get_leaderboard

import mlflow
import mlflow.h2o
from mlflow.tracking import MlflowClient

import json

In [2]:
# Initiate H2O cluster
h2o.init()

# Initiate MLflow client
client = MlflowClient()

# Get parsed experiment name
experiment_name = 'experiment2'

# Create MLflow experiment
try:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = client.get_experiment_by_name(experiment_name)
except:
    experiment = client.get_experiment_by_name(experiment_name)

mlflow.set_experiment(experiment_name)

# Print experiment details
print(f"Name: {experiment_name}")
print(f"Experiment_id: {experiment.experiment_id}")
print(f"Artifact Location: {experiment.artifact_location}")
print(f"Lifecycle_stage: {experiment.lifecycle_stage}")
print(f"Tracking uri: {mlflow.get_tracking_uri()}")


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,53 secs
H2O_cluster_timezone:,Europe/Bucharest
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,9 days
H2O_cluster_name:,H2O_from_python_robert_radoslav_lggrgs
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Name: experiment2
Experiment_id: 2
Artifact Location: file:///Users/robert.radoslav/Developer/main/finpy-containerized-app/finpy-api/ml_models/mlruns/2
Lifecycle_stage: active
Tracking uri: file:///Users/robert.radoslav/Developer/main/finpy-containerized-app/finpy-api/ml_models/mlruns


In [4]:
# Import data directly as H2O frame (default location is data/processed)
main_frame = h2o.import_file(path='health_insurance/data/processed/train.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
with open('health_insurance/data/processed/train_col_types.json', 'w') as fp:
        json.dump(main_frame.types, fp)

In [6]:
target = 'Response'
predictors = [n for n in main_frame.col_names if n != target]

In [9]:
with mlflow.start_run():
        aml = H2OAutoML(
                        max_models=5, # Run AutoML for n base models
                        seed=42, 
                        balance_classes=True, # Target classes imbalanced, so set this as True
                        sort_metric='logloss', # Sort models by logloss (metric for multi-classification)
                        verbosity='info', # Turn on verbose info
                        exclude_algos = ['GLM', 'DRF'], # Specify algorithms to exclude
                    )
        
        # Initiate AutoML training
        aml.train(x=predictors, y=target, training_frame=main_frame)
        
        # Set metrics to log
        mlflow.log_metric("log_loss", aml.leader.logloss())
        mlflow.log_metric("AUC", aml.leader.auc())
        
        # Log and save best model (mlflow.h2o provides API for logging & loading H2O models)
        mlflow.h2o.log_model(aml.leader, artifact_path="model")
        
        model_uri = mlflow.get_artifact_uri("model")
        print(f'AutoML best model saved in {model_uri}')
        
        # Get IDs of current experiment run
        exp_id = experiment.experiment_id
        run_id = mlflow.active_run().info.run_id
        
        # Save leaderboard as CSV
        lb = get_leaderboard(aml, extra_columns='ALL')
        lb_path = f'health_insurance/mlruns/{exp_id}/{run_id}/artifacts/model/leaderboard.csv'
        lb.as_data_frame().to_csv(lb_path, index=False) 
        print(f'AutoML Complete. Leaderboard saved in {lb_path}')

AutoML progress: |
01:48:09.237: Project: AutoML_1_20230218_14809
01:48:09.241: 5-fold cross-validation will be used.
01:48:09.241: Setting stopping tolerance adaptively based on the training frame: 0.0033065124063248246
01:48:09.241: Build control seed: 42
01:48:09.243: training frame: Frame key: AutoML_1_20230218_14809_training_train.hex    cols: 208    rows: 91466  chunks: 32    size: 2477845  checksum: -277864416819399133
01:48:09.243: validation frame: NULL
01:48:09.243: leaderboard frame: NULL
01:48:09.243: blending frame: NULL
01:48:09.243: response column: Response
01:48:09.243: fold column: null
01:48:09.243: weights column: null
01:48:09.261: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (7g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_1 (4g, 60w), lr_annealing (7g, 10w)]}, {De

OSError: Job with key $03017f00000132d4ffffffff$_bfa9e39404193b0a21e2dff38b564c7e failed with an exception: water.exceptions.H2OAutoMLException: Aborting AutoML after too many consecutive model failures
stacktrace: 
water.exceptions.H2OAutoMLException: Aborting AutoML after too many consecutive model failures
	at ai.h2o.automl.AutoML.learn(AutoML.java:776)
	at ai.h2o.automl.AutoML.run(AutoML.java:494)
	at ai.h2o.automl.H2OJob$1.compute2(H2OJob.java:33)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1677)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:976)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1479)
	at jsr166y.ForkJoinWorkerThread.run(ForkJoinWorkerThread.java:104)
